In [7]:
import os
import torch.multiprocessing as mp
import torch.distributed as dist
import torch

def distributed_init():

    dist.init_process_group(backend="nccl")
    local_rank = dist.get_rank()
    world_size = dist.get_world_size()
    torch.cuda.set_device(local_rank)

    return local_rank, world_size

In [8]:
def find_free_port():
    """ https://stackoverflow.com/questions/1365265/on-localhost-how-do-i-pick-a-free-port-number """
    import socket
    from contextlib import closing

    with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
        s.bind(('', 0))
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        return str(s.getsockname()[1])

In [12]:
world_size = 2
master_addr = '127.0.0.1'
master_port = find_free_port()

In [13]:
master_port

'57727'

In [ ]:
from datetime import timedelta

rank = 0
backend = 'nccl'

print(f'setting up {rank=} {world_size=} {backend=}')

# set up the master's ip address so this child process can coordinate
os.environ['MASTER_ADDR'] = master_addr
os.environ['MASTER_PORT'] = master_port
print(f"{master_addr=} {master_port=}")

# Initializes the default distributed process group, and this will also initialize the distributed package.
dist.init_process_group(backend, rank=rank, world_size=world_size, timeout=timedelta(seconds=30), init_method="env://")
print(f"{rank=} init complete")
dist.destroy_process_group()
print(f"{rank=} destroy complete")

In [ ]:
CUDA_VISIBLE_DEVICES=6,7 OMP_NUM_THREADS=48 torchrun --nproc_per_node=2 test/TP_baseline.py

In [5]:
for i in range(3, 100):
    print(f"CUDA_VISIBLE_DEVICES=0 python test/e2e_ablation.py --prefill {122880} --budget {128*i} --chunk_size 8 --top_p 0.9 --temp 0.6 --gamma 6")

CUDA_VISIBLE_DEVICES=0 python test/e2e_ablation.py --prefill 122880 --budget 384 --chunk_size 8 --top_p 0.9 --temp 0.6 --gamma 6
CUDA_VISIBLE_DEVICES=0 python test/e2e_ablation.py --prefill 122880 --budget 512 --chunk_size 8 --top_p 0.9 --temp 0.6 --gamma 6
CUDA_VISIBLE_DEVICES=0 python test/e2e_ablation.py --prefill 122880 --budget 640 --chunk_size 8 --top_p 0.9 --temp 0.6 --gamma 6
CUDA_VISIBLE_DEVICES=0 python test/e2e_ablation.py --prefill 122880 --budget 768 --chunk_size 8 --top_p 0.9 --temp 0.6 --gamma 6
CUDA_VISIBLE_DEVICES=0 python test/e2e_ablation.py --prefill 122880 --budget 896 --chunk_size 8 --top_p 0.9 --temp 0.6 --gamma 6
CUDA_VISIBLE_DEVICES=0 python test/e2e_ablation.py --prefill 122880 --budget 1024 --chunk_size 8 --top_p 0.9 --temp 0.6 --gamma 6
CUDA_VISIBLE_DEVICES=0 python test/e2e_ablation.py --prefill 122880 --budget 1152 --chunk_size 8 --top_p 0.9 --temp 0.6 --gamma 6
CUDA_VISIBLE_DEVICES=0 python test/e2e_ablation.py --prefill 122880 --budget 1280 --chunk_size 

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("JackFram/llama-68m")

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [19]:
tokenizer.decode([263, 1568,  263,  590,  777,  263])

'a much a my some a'

In [24]:
bsz_list = [1, 2, 4, 8, 16, 32, 64, 128]
cuda_list = ['1', '1,2', '1,2,3,4', '1,2,3,4,5,6,8,9']

# CUDA_VISIBLE_DEVICES=5,6 OMP_NUM_THREADS=48 torchrun --nproc_per_node=2 test/TP_baseline.py --prefill 128 --bsz 1 --gen_len 32

for cuda in reversed(cuda_list):
    for bsz in reversed(bsz_list):
        print(f"CUDA_VISIBLE_DEVICES={cuda} OMP_NUM_THREADS=48 torchrun --nproc_per_node={len(cuda)//2+1} test/TP_baseline.py --prefill 128 --bsz {bsz} --gen_len 32")

CUDA_VISIBLE_DEVICES=1,2,3,4,5,6,8,9 OMP_NUM_THREADS=48 torchrun --nproc_per_node=8 test/TP_baseline.py --prefill 128 --bsz 128 --gen_len 32
CUDA_VISIBLE_DEVICES=1,2,3,4,5,6,8,9 OMP_NUM_THREADS=48 torchrun --nproc_per_node=8 test/TP_baseline.py --prefill 128 --bsz 64 --gen_len 32
CUDA_VISIBLE_DEVICES=1,2,3,4,5,6,8,9 OMP_NUM_THREADS=48 torchrun --nproc_per_node=8 test/TP_baseline.py --prefill 128 --bsz 32 --gen_len 32
CUDA_VISIBLE_DEVICES=1,2,3,4,5,6,8,9 OMP_NUM_THREADS=48 torchrun --nproc_per_node=8 test/TP_baseline.py --prefill 128 --bsz 16 --gen_len 32
CUDA_VISIBLE_DEVICES=1,2,3,4,5,6,8,9 OMP_NUM_THREADS=48 torchrun --nproc_per_node=8 test/TP_baseline.py --prefill 128 --bsz 8 --gen_len 32
CUDA_VISIBLE_DEVICES=1,2,3,4,5,6,8,9 OMP_NUM_THREADS=48 torchrun --nproc_per_node=8 test/TP_baseline.py --prefill 128 --bsz 4 --gen_len 32
CUDA_VISIBLE_DEVICES=1,2,3,4,5,6,8,9 OMP_NUM_THREADS=48 torchrun --nproc_per_node=8 test/TP_baseline.py --prefill 128 --bsz 2 --gen_len 32
CUDA_VISIBLE_DEVICES=1